In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

# Use the citipy module to determine city based on latitude and longitude.The citipy module finds the nearest city to 
# the latitude and longitude pair with a population of 500 or more.
from citipy import citipy

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

# Import the datetime module from the datetime library.
from datetime import datetime

NameError: name 'gmaps' is not defined

In [ ]:
# Create a set of random latitude and longitude combinations. lats = lattitudes, lngs =  longitudes. zip saves it as a zip
# object which pachs each pair of lats and lngs having the same index in their respective array into a tuple.  If there are
# 1,500 lats and lngs, there will be 1,500 tuples of paired lats and lngs, where each lats and lngs in a tuple can be
# accessed by the index of 0 and 1, respectively. 
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [ ]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [ ]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

# Some of this code should look familiar, but let's break it down:

# We create a cities list to store city names.
# We iterate through the coordinates, as in our practice, and retrieve the nearest city using the lats and longs pair.
# We add a decision statement with the logical operator not in to determine whether the found city is already in the cities
# list. If not, then we'll use the append() function to add it. We are doing this because among the 1,500 latitudes and 
# longitudes, there might be duplicates, which will retrieve duplicate cities, and we want to be sure we capture only the 
# unique cities.

In [ ]:
# For each city in our lats_lngs list, we need to retrieve the following data and add it to a DataFrame:

    # City, country, and date
    # Latitude and longitude
    # Maximum temperature
    # Humidity
    # Cloudiness
    # Wind speed

# Let's use pseudocode to map out, at a high level, how we will get the weather data for each city for the website.

# We will need to do the following:

# import our dependencies and initialize counters and an empty list that will hold the weather data. Dependencies will be added above
# Loop through the cities list.
# Group the cities in sets of 50 to log the process as we find the weather data for each city.
# Two counters will be needed here: one to log the city count from 1 to 50, and another for the sets.
# Build the city_url or endpoint for each city.
# Log the URL and the record and set numbers.
# Make an API request for each city.
# Parse the JSON weather data for the following:
# City, country, and date
# Latitude and longitude
# Maximum temperature
# Humidity
# Cloudiness
# Wind speed
# Add the data to a list in a dictionary format and then convert the list to a DataFrame.

# First create teh basic URL for the OpenWeatherMap
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [ ]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list. This creates the URL for each one and groups our records in sets of 50.
# enumerate can be used instead of creating to for loops
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Let's break down the code so we understand fully before continuing:

    # We create the for loop with the enumerate() method and reference the index and the city in the list.
    # In the conditional statement, we check if the remainder of the index divided by 50 is equal to 0 and if the index 
    # is greater than or equal to 50. If the statement is true, then the set_count and the record_count are incremented by 1.
    # Inside the conditional statement, we create the URL endpoint for each city, as before. However, we are removing the 
    # blank spaces in the city name and concatenating the city name with, city.replace(" ","+"). This will find the 
    # corresponding weather data for the city instead of finding the weather data for the first part of the city name.
    # Also, we add a print statement that tells us the record count and set count, and the city that is being processed.
    # Then we add one to the record count before the next city is processed.
    
# Run an API request for each of the cities. If there is not data for the city, i.e., a response 404 then there is not 
# weather to retrieve and city not found. skipping is printed. Pass is a general purpose statem to handle errors and 
# allow the program to continue. Usually we dont want to use this and instead add another try block to print the error. 

    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                    "Lat": city_lat,
                    "Lng": city_lng,
                    "Max Temp": city_max_temp,
                    "Humidity": city_humidity,
                    "Cloudiness": city_clouds,
                    "Wind Speed": city_wind,
                    "Country": city_country,
                    "Date": city_date})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

In [ ]:
# Change the column order
new_column_order  = ['City','Country','Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed',]

# Assign district summary df the new column order.
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)


In [ ]:
#Save dataframe as csv
pd.DataFrame.to_csv(city_data_df, 'cities.csv',)
city_data_df.head()

In [ ]:
# Extract relevant fields from the DataFrame for plotting.
lats = city_data_df["Lat"]
max_temps = city_data_df["Max Temp"]
humidity = city_data_df["Humidity"]
cloudiness = city_data_df["Cloudiness"]
wind_speed = city_data_df["Wind Speed"]

In [ ]:
# Import time module
import time

# Build the scatter plot for latitude vs. max temperature.
plt.scatter(lats,
            max_temps,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Max Temperature "+ time.strftime("%x"))
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure.
plt.savefig("weather_data/Fig1.png")

# Show plot.
plt.show()

In [ ]:
# Build the scatter plots for latitude vs. humidity.
plt.scatter(lats,
            humidity,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Humidity "+ time.strftime("%x"))
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.grid(True)
# Save the figure.
plt.savefig("weather_data/Fig2.png")
# Show plot.
plt.show()

In [ ]:
# Build the scatter plots for latitude vs. cloudiness.
plt.scatter(lats,
            cloudiness,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Cloudiness (%) "+ time.strftime("%x"))
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)
# Save the figure.
plt.savefig("weather_data/Fig3.png")
# Show plot.
plt.show()

In [ ]:
# Build the scatter plots for latitude vs. wind speed.
plt.scatter(lats,
            wind_speed,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Wind Speed "+ time.strftime("%x"))
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude")
plt.grid(True)
# Save the figure.
plt.savefig("weather_data/Fig4.png")
# Show plot.
plt.show()

In [ ]:
# Import linear regression from the SciPy stats module. We need to do this to run linear regression.
from scipy.stats import linregress

In [ ]:
# Create an equal number of latitudes and temperatures.
lats = [42.5, 43.9, 8.1, 36.8, 79.9, 69.1, 25.7, 15.3, 12.7, 64.5]
temps = [80.5, 75.3, 90.9, 90.0, 40.4, 62.3, 85.4, 79.6, 72.5, 72.0]

In [ ]:
# you can use the linregress function to calculate slope, y-intercept, correlation coefficient (r-value), p-value
# and standard deviation

# Perform linear regression.
(slope, intercept, r_value, p_value, std_err) = linregress(lats, temps)
# Get the equation of the line.
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
print(line_eq)
print(f"The p-value is: {p_value:.3f}")

# Slope of the regression line as slope
# y-intercept as intercept
# Correlation coefficient as r_value
# p-value as p_value
# Standard error as std_err

# In statistics, the p-value is used to determine significance of results. In most cases, data scientists like to use a 
# significance level of 0.05, which means:

# A linear regression with a p-value > 0.05 is not statistically significant.

# A linear regression with a p-value < 0.05 is statistically significant.

# P-values can also be used to justify rejecting a null hypothesis. 

In [ ]:
# Calculate the ideal temperatures (y-values) using the slope and intercept from the equation of the regression line.

# Calculate the regression line "y values" from the slope and intercept.
regress_values = [(lat * slope + intercept) for lat in lats]

In [ ]:
# Import Matplotlib.
import matplotlib.pyplot as plt
# Create a scatter plot of the x and y values.
plt.scatter(lats,temps)
# Plot the regression line with the x-values and the y coordinates based on the intercept and slope.
plt.plot(lats,regress_values,"r")
# Annotate the text for the line equation and add its coordinates.
plt.annotate(line_eq, (10,40), fontsize=15, color="red")
plt.xlabel('Latitude')
plt.ylabel('Temp')
plt.show()


# Let's review what this code does:

# We plot the latitudes and temperatures on a scatter plot.
# We create a line plot of our regression line with the ideal temperatures.
# We annotate the line plot by adding the equation of our regression line, where the x-axis is 10 and the y-axis is 40, 
# and specify the font and color.
# We create x- and y-axes labels.

In [ ]:
# Import linregress
from scipy.stats import linregress

# Create a function to create perform linear regression on the weather data
# and plot a regression line and the equation with the data.
def plot_linear_regression(x_values, y_values, title, y_label, text_coordinates):

    # Run regression on hemisphere weather data.
    (slope, intercept, r_value, p_value, std_err) = linregress(x_values, y_values)

    # Calculate the regression line "y values" from the slope and intercept.
    regress_values = x_values * slope + intercept
    # Get the equation of the line.
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    # Create a scatter plot and plot the regression line.
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r")
    # Annotate the text for the line equation.
    plt.annotate(line_eq, text_coordinates, fontsize=15, color="red")
    plt.xlabel('Latitude')
    plt.ylabel(y_label)
    plt.show()

In [ ]:
# Create the hemisphere dataframes
# To create a new DataFrame from a current DataFrame, we can use the loc method on the current DataFrame. The loc method 
# accesses a group of rows and columns in the current DataFrame by an index, labels, or a Boolean array. The syntax to get 
# a specific row from a current DataFrame is row = df.loc[row_index].

index13 = city_data_df.loc[13]
index13

In [ ]:
# We can also filter a DataFrame based on a value of a row. For instance, if we wanted to get all Northern Hemisphere 
# latitudes, for latitudes greater than or equal to 0, we can filter the city_data_df DataFrame using the 
# code city_data_df["Lat"] >= 0. Executing this code will return either "True" or "False" for all the rows that meet these 
# criteria.

city_data_df["Lat"] >= 0

In [ ]:
# If we want to return a DataFrame with all data fitting the criteria, for latitudes greater than or equal to 0, we can
# use the loc method on the city_data_df DataFrame. Inside the brackets, we would add the conditional 
# filter city_data_df["Lat"] >= 0 so that our statement would appear as:

city_data_df.loc[(city_data_df["Lat"] >= 0)].head()

# Also, since this is a DataFrame, we can add the head() method at the end to get the first five rows, not counting the 
# row of column headings

In [ ]:
# Now assign this DataFrame to the variable northern_hemi_df to access the data to perform linear regression.

# We can take the same approach to get the cities for the Southern Hemisphere by filtering the city_data_df DataFrame for 
# latitudes less than 0.

# To create DataFrames for the Northern and Southern Hemispheres' data, add the code to a new cell and run the code.

# Create Northern and Southern Hemisphere DataFrames.
northern_hemi_df = city_data_df.loc[(city_data_df["Lat"] >= 0)]
southern_hemi_df = city_data_df.loc[(city_data_df["Lat"] < 0)]

# Now we can perform linear regression on latitude and maximum temperature from each hemisphere DataFrame.

In [ ]:
# To generate the linear regression on the maximum temperature for the Northern Hemisphere, we'll need x and y values. 
# Set the x values equal to the latitude column and the y values equal to the maximum temperature column from the
# northern_hemi_df DataFrame.

# Call the plot_linear_regression function with the x and y values, and edit the title, y_label, and text_coordinates for
# the maximum temperature scatter plot. Add the code to a new cell and run it to generate the linear regression and plot 
# the data.

# Linear regression on the Northern Hemisphere
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Max Temp"]
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Northern Hemisphere \
                        for Maximum Temperature', 'Max Temp',(10,40))

# Note: If the equation for the regression line doesn't show up on your graph, you can change the text_coordinates until 
# you see the equation.

In [ ]:
# generate the linear regression on the maximum temperature for the Southern Hemisphere
# Linear regression on the Southern Hemisphere
x_values = southern_hemi_df["Lat"]
y_values = southern_hemi_df["Max Temp"]



# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Southern Hemisphere \
                        for Maximum Temperature', 'Max Temp',(-50,90))




In [ ]:
# To perform the linear regression on the percent humidity for the Northern Hemisphere, set the x-value equal to the 
# latitude column and y-value equal to the Humidity column from the northern_hemi_df DataFrame.

# Call the plot_linear_regression function with the x- and y-values, and edit the title, y_label, and text_coordinates 
# for the percent humidity scatter plot.


# Linear regression on the Northern Hemisphere
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Humidity"]
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Northern Hemisphere \
                        for % Humidity', '% Humidity',(40,10))

# Findings:
# The correlation between the latitude and percent humidity is very low because the r-value is less than 0.04 for the
# Northern and Southern Hemispheres for the plots shown here. This means that percent humidity is unpredictable due to
# changing weather patterns that can increase or decrease percent humidity. Check the r-values for your plots. 

In [ ]:
# Linear regression on the Southern Hemisphere 6.4.3 This doesnt even have the formula. Need to learn how to adjust formula positon
x_values = southern_hemi_df["Lat"]
y_values = southern_hemi_df["Humidity"]


# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Southern Hemisphere \
                        for % Humidity', '% Humidity',(-50,15))




In [ ]:
# Perform Linear Regression on the Wind Speed for the Northern Hemisphere
# Call the plot_linear_regression function, with the x-value equal to the Latitude column and the y-value equal to the 
# Wind Speed column from the northern_hemi_df DataFrame. Edit the title, y_label, and text_coordinates for the wind speed 
# scatter plot.

# Linear regression on the Northern Hemisphere
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Wind Speed"]
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Northern Hemisphere \
                        for Wind Speed', 'Wind Speed',(40,35))

In [ ]:
# Perform Linear Regression on the Wind Speed for the Southern Hemisphere
# Finally, let's call the plot_linear_regression function, with the x-value equal to the latitude column and the y-value 
# equal to wind speed column from the southern_hemi_df DataFrame. Edit the title, y_label, and text_coordinates for the
# wind speed scatter plot.

# Linear regression on the Southern Hemisphere
x_values = southern_hemi_df["Lat"]
y_values = southern_hemi_df["Wind Speed"]
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Southern Hemisphere \
                        for Wind Speed', 'Wind Speed',(-50,35))

In [ ]:
# Create Heatmaps for Weather Parameters
# Begin by importing our dependencies and Google API key, and then add our cities.csv file to a DataFrame. 
# Create a new Jupyter Notebook file named VacationPy.ipynb. In the first cell, add our dependencies and API key.

# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [ ]:
# Next, we'll read our cities.csv file into a DataFrame.
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

In [ ]:
# Gett the data type for the above csv 
city_data_df.dtypes

In [ ]:
# Create a Maximum Temperature Heatmap
# First, tell gmaps to use your API key. You only need to configure gmaps to use your API key once.
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [ ]:
# Google heatmaps do not plot negative numbers. If you have a maximum temperature that is less than 0 °F, then 
# you will get an InvalidWeightException error To remove the negative temperatures we can use a for loop to iterate 
# through the max_temp and add the temperatures that are greater than 0 °F to a new list.

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [ ]:
# Next, create the heatmap for the maximum temperature. The general syntax for creating a heatmap is as follows:

# 1. Assign the locations to an array of latitude and longitude pairs.
#locations = [latitude, longitude]
# 2. Assign the weights variable to some values.
#temperatures = # an array of length equal to the locations array length
# 3. Assign the figure variable to the gmaps.figure() attribute.
#fig = gmaps.figure()
# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
#heatmap_layer = gmaps.heatmap_layer(locations, weights=temperatures)

# 5. Add the heatmap layer.
#fig.add_layer(heatmap_layer)
# 6. Call the figure to plot the data.
#fig

# Heatmap of temperature This is the actual one
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.

In [ ]:
# Create a Percent Humidity Heatmap

# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

In [ ]:
# To create the heatmap for percent cloudiness, replace the measurement from the humidity values to cloudiness values.


# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

In [ ]:
# Create a Wind Speed Heatmap


# Heatmap of percent wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig



In [ ]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [ ]:
# filter the maximum temperature column in the city_data_df DataFrame using logical operators to create a new DataFrame 
# with the cities that meet the customer's criteria.Recall that you use the loc[] method on the current DataFrame to 
# create a new DataFrame from a current DataFrame. Inside the brackets, add the conditional filter.
                                         
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)   

In [ ]:
preferred_cities_df.count()

In [ ]:
# Once the customers have filtered the database (DataFrame) based on their temperature preferences, show them a
# heatmap for the maximum temperature for the filtered cities. In addition, create a marker for each city that 
# will display the name of the city, country code, maximum temperature, and name of a nearby hotel within three miles
# of the coordinates when the marker is clicked.
# Don't add the hotel information to the preferred_cities_df DataFrame because this DataFrame is our filtered
# DataFrame, and the customer will always filter it for each trip. We'll need to create a new DataFrame specifically 
# for the data needed to create a heatmap and pop-up markers
# Make a copy of the preferred_cities_df DataFrame and name it hotel_df. For the hotel_df, keep the 
# columns "City," "Country," "Max Temp," "Lat," and "Lng." Add a new column to the hotel_df DataFrame to hold the 
# name of the hotel.

# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

hotel_df.head(10)

In [ ]:
# Retrieve Hotels from a Nearby Search
# The first step for retrieving hotels from a Nearby Search is to set the parameters for the search.

# Set the Parameters for a Nearby Search
# To find the nearest establishment to geographic coordinates, use the Google Places Nearby Search request. 
# First, navigate to the Nearby Search requests page.

# The Nearby Search request lets us perform these tasks:

# Search for places within a specified area.
# Refine the search using keywords and specifying the type of place we are searching for.
# Use an API URL, where the output can be either XML or JSON format.

 # The documentation states we must provide specific parameters.
    # Specific parameters include:

# Our API key
# A location, which will be the latitude and longitude
# A radius for the search. The radius can be up to 50,000 meters or approximately 31 miles. This distance is much too great for finding hotels, so we'll keep our search to 5,000 meters.
# If we use the rankby=distance for a parameter, we need to add one or more of the three parameters above. We won't use the rankby=distance because we will be search based on the radius from a given latitude and longitude.
# We can add optional parameters such as a keyword, a type of business, or the name of a business.

# For our hotel search, we'll use these parameters:

# API key
# Latitude and longitude
# 5,000-meter radius
# Type of place

# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}